In [1]:
!tree ../dataset

../dataset
├── en_espanol
│   ├── docx2txt.py
│   ├── Второй_жанр_исходная.txt
│   └── Первый_жанр_исходная.txt
├── Второй_жанр_исходная.txt
├── Первый_жанр_исходная.txt
├── Сокращение по частям речи
│   ├── 1.Первый жанр исходная выборка.txt
│   ├── 2.Первый жанр без клауз, включающих наречия.txt
│   ├── 3.Первый жанр без клауз, включающих глаголы.txt
│   ├── 4.Первый жанр без клауз, включающих глаголы и наречия.txt
│   ├── Без прилагательных второй жанр.txt
│   ├── Без прилагательных первый жанр.txt
│   └── Случайные выборки.txt
└── сокращение по частотности
    ├── 1а_ без сокращений.txt
    ├── 1б_Изъяты лексемы с частотой выше 100.txt
    ├── 1в_Изъяты лексемы с частотой выше 49.txt
    ├── 1г_Изъяты лексемы с частотой выше 29.txt
    ├── 1д_Изъяты лексемы с частотой выше 9.txt
    ├── 1е_Изъяты лексемы с частотой выше 5.txt
    ├── 1ё_Изъяты лексемы с частотой выше 3.txt
    ├── 2а_ без сокращений.txt
    ├── 2б_Изъяты лексемы с частотой выше 100.txt
    ├── 2в_Изъяты лексемы с ч

In [2]:
ls

 Untitled.ipynb
 __pycache__/
 accuracy_by_dataset_type.png
 comprehensive_comparison_freq.png
 comprehensive_comparison_pos.png
 confronto_prestazioni.png
'confusion_matrices_gpt2_1.Первый жанр исходная выборка.png'
'confusion_matrices_gpt2_Изъяты лексемы с частотой выше 100.png'
 confusion_matrix_DeepPavlov_rubert-base-cased_Изъяты_лексемы_с_частотой_выше_100.png
 confusion_matrix_bert-base-multilingual-cased_Изъяты_лексемы_с_частотой_выше_100.png
 confusion_matrix_distilbert-base-multilingual-cased_Изъяты_лексемы_с_частотой_выше_100.png
 confusion_matrix_facebook_opt-125m_1.Первый_жанр_исходная_выборка.png
 confusion_matrix_facebook_opt-125m_Изъяты_лексемы_с_частотой_выше_100.png
 confusion_matrix_gpt2_1.Первый_жанр_исходная_выборка.png
 confusion_matrix_gpt2_Изъяты_лексемы_с_частотой_выше_100.png
 confusion_matrix_roberta-base_Изъяты_лексемы_с_частотой_выше_100.png
 confusion_matrix_sberbank-ai_rugpt3small_based_on_gpt2_1.Первый_жанр_исходная_выборка.png
 confusion_matrix_sberbank-

In [3]:
import json

def save_results(results, filename):
    with open(filename, 'w', encoding='utf-8') as f:
        json.dump(results, f, indent=4, ensure_ascii=False)

In [ ]:
import funciones
from utils import train_wrapper
import warnings
import os
import torch
import matplotlib.pyplot as plt
from concurrent.futures import ProcessPoolExecutor
from tqdm import tqdm
from contextlib import redirect_stderr
import multiprocessing as mp
import numpy as np
#import wandb
import nbformat
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from sklearn.metrics import classification_report, confusion_matrix
import gc
import re
import nltk
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from transformers import BertForSequenceClassification, AdamW
from razdel import sentenize
import numpy as np

# Suprimir warnings específicos
warnings.filterwarnings('ignore', category=UserWarning)
warnings.filterwarnings('ignore', category=FutureWarning)
mp.set_start_method('spawn', force=True)


import json


def convert_numpy(obj):
    if isinstance(obj, np.ndarray):
        return obj.tolist()
    if isinstance(obj, (np.float32, np.float64, np.int32, np.int64)):
        return obj.item()
    return obj

def save_results(results, filename):
    with open(filename, 'w', encoding='utf-8') as f:
        json.dump(results, f, indent=4, ensure_ascii=False, default=convert_numpy)

# import funciones
# from utils import train_wrapper
# import warnings
# import os
# import torch
# import matplotlib.pyplot as plt

# # Suprimir warnings específicos
# warnings.filterwarnings('ignore', category=UserWarning)  # Para sklearn y otros
# warnings.filterwarnings('ignore', category=FutureWarning)  # Para huggingface y transformers
# from concurrent.futures import ProcessPoolExecutor
# from tqdm import tqdm
# from contextlib import redirect_stderr
# from concurrent.futures import ProcessPoolExecutor
# import multiprocessing as mp
# import numpy as np
# mp.set_start_method('spawn', force=True)
# #os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "max_split_size_mb:128"

###############################################################################
# import wandb
# import nbformat
# # Configurar la clave de la API como variable de entorno
# os.environ["WANDB_API_KEY"] = "ca85316ed713b2425615fb3a613d7eb414c9f57f"  # Reemplaza con tu clave
# # Iniciar wandb sin especificar entity (se detecta automáticamente)
# wandb.init(project="model-clasification")
# wandb.init(
#     settings=wandb.Settings(
#         start_method="thread",
#         timeout=30,
#         sync_dir="/tmp/wandb",  # Usa un directorio temporal
#         disable_code=True       # Mejora la estabilidad
#     )
# )
#os.environ["WANDB_NOTEBOOK_NAME"] = "сокращение по частотности.ipynb"
###############################################################################


def create_custom_config(model_name, model_type, dataset):
    """Crea configuración de entrenamiento adaptativa"""
    common_config = {
#         'dataset_name':dataset.get('name', ''),
        'model_name': model_name,
        'model_type': model_type,
        'num_repeats': 6,
        'test_size': 0.2,
        'threshold': 0.5
    }

    # Configuración específica para GPT
    if model_type == 'gpt':
        return funciones.TrainingConfig(
            **common_config,
            max_length=52,
            batch_size=64,
            epochs=6,
            learning_rate=1e-5
        )
#         freq_threshold = dataset.get('freq_threshold')
#         if freq_threshold in [100, 49, 29, 9, 5, 3]:
#             configs = {
#                 100: (52, 32, 6, 1e-5),#test 1
#                 49: (60, 128, 4, 3e-5),
#                 29: (51, 128, 4, 3e-5),
#                 9: (45, 128, 5, 4e-5),
#                 5: (150, 32, 6, 5e-5),
#                 3: (150, 32, 6, 5e-5)
#             }
#             max_len, batch, epochs, lr = configs[freq_threshold]
#             return funciones.TrainingConfig(
#                 **common_config,
#                 max_length=max_len,
#                 batch_size=batch,
#                 epochs=epochs,
#                 learning_rate=lr
#             )

        # Configuración por nombre de dataset
        dataset_name = dataset.get('name', '')
        if any(x in dataset_name for x in ['1', '2', '3', '4']):
            return funciones.TrainingConfig(
                **common_config,
                max_length=60,
                batch_size=128,
                epochs=6,
                learning_rate=5e-5
            )


    # Configuraciones basadas en frecuencia
    freq_threshold = dataset.get('freq_threshold')
    if freq_threshold in [100, 49, 29, 9, 5, 3]:
        configs = {
            100: (52, 128, 3, 2e-5),
            49: (60, 128, 4, 3e-5),
            29: (51, 128, 4, 3e-5),
            9: (45, 128, 5, 4e-5),
            5: (150, 32, 6, 5e-5),
            3: (150, 32, 6, 5e-5)
        }
        max_len, batch, epochs, lr = configs[freq_threshold]
        return funciones.TrainingConfig(
            **common_config,
            max_length=max_len,
            batch_size=batch,
            epochs=epochs,
            learning_rate=lr
        )

    # Configuración por nombre de dataset
    dataset_name = dataset.get('name', '')
    if any(x in dataset_name for x in ['1', '2', '3', '4']):
        return funciones.TrainingConfig(
            **common_config,
            max_length=60,
            batch_size=128,
            epochs=6,
            learning_rate=5e-5
        )

    # Configuración por defecto
    return funciones.TrainingConfig(
        **common_config,
        max_length=60,
        batch_size=32,
        epochs=4,
        learning_rate=3e-5
    )

models = [
        {'model':'DeepPavlov/rubert-base-cased',
         'name':'DeepPavlov-rubert-base',
         'type': 'bert'},# Modelo original
        {'model':'bert-base-multilingual-cased',
         'name':'BERT multilingual',
         'type': 'bert'},# BERT multilingüe
        {'model':'distilbert-base-multilingual-cased',
         'name':'distilbert-base-multilingual',
         'type': 'bert'},# Versión ligera de BERT
        {'model':'roberta-base',
         'name':'roberta-base', 
        'type': 'bert'}, # RoBERTa 
        {'model': 'gpt2',
         'name': 'gpt2',
         'type': 'gpt'},  #  GPT model
        {'model': 'facebook/opt-125m',
         'name': 'facebook',
         'type': 'gpt'},  #  GPT model
        {'model': 'sberbank-ai/rugpt3small_based_on_gpt2',
         'name': 'rugpt3',
         'type': 'gpt'}
    ]

datasets = [
        {
            'path1': '../dataset/сокращение по частотности/1б_Изъяты лексемы с частотой выше 100.txt',
            'path2': '../dataset/сокращение по частотности/2б_Изъяты лексемы с частотой выше 100.txt',
            'name': 'Изъяты лексемы с частотой выше 100',
            'type': 'freq',
            'freq': 100
        },
#         {
#             'path1': '../dataset/сокращение по частотности/1в_Изъяты лексемы с частотой выше 49.txt',
#             'path2': '../dataset/сокращение по частотности/2в_Изъяты лексемы с частотой выше 49.txt',
#             'name': 'Изъяты лексемы с частотой выше 49',
#             'type': 'freq',
#             'freq': 49
#         },
#         {
#             'path1': '../dataset/сокращение по частотности/1г_Изъяты лексемы с частотой выше 29.txt',
#             'path2': '../dataset/сокращение по частотности/2г_Изъяты лексемы с частотой выше 29.txt',
#             'name': 'Изъяты лексемы с частотой выше 29',
#             'type': 'freq',
#             'freq': 29
#         },
#         {
#             'path1': '../dataset/сокращение по частотности/1д_Изъяты лексемы с частотой выше 9.txt',
#             'path2': '../dataset/сокращение по частотности/2д_Изъяты лексемы с частотой выше 9.txt',
#             'name': 'Изъяты лексемы с частотой выше 9',
#             'type': 'freq',
#             'freq': 9
#         },
#         {
#             'path1': '../dataset/сокращение по частотности/1е_Изъяты лексемы с частотой выше 5.txt',
#             'path2': '../dataset/сокращение по частотности/2е_Изъяты лексемы с частотой выше 5.txt',
#             'name': 'Изъяты лексемы с частотой выше 5',
#             'type': 'freq',
#             'freq': 5
#         },
#         {
#             'path1': '../dataset/сокращение по частотности/1ё_Изъяты лексемы с частотой выше 3.txt',
#             'path2': '../dataset/сокращение по частотности/2ё_Изъяты лексемы с частотой выше 3.txt',
#             'name': 'Изъяты лексемы с частотой выше 3',
#             'type': 'freq',
#             'freq': 3
#         },

#             {
#             'path1': '../dataset/Сокращение по частям речи/Без прилагательных первый жанр.txt',
#             'path2': '../dataset/Сокращение по частям речи/Без прилагательных второй жанр.txt',
#             'name': 'Без прилагательных второй жанр',
#             'type': 'pos',
#             'freq': None
#             },
          {
            'path1': '../dataset/Сокращение по частям речи/1.Первый жанр исходная выборка.txt',
            'path2': '../dataset/Второй_жанр_исходная.txt',
            'name': '1.Первый жанр исходная выборка',
            'type': 'pos',
            'freq': None

        },
#         {
#             'path1': '../dataset/Сокращение по частям речи/2.Первый жанр без клауз, включающих наречия.txt',
#             'path2': '../dataset/Второй_жанр_исходная.txt',
#             'name': '2.Первый жанр без клауз, включающих наречия',
#             'type': 'pos',
#             'freq': None
#         },
#         {
#             'path1': '../dataset/Сокращение по частям речи/3.Первый жанр без клауз, включающих глаголы.txt',
#             'path2': '../dataset/Второй_жанр_исходная.txt',
#             'name': '3.Первый жанр без клауз, включающих глаголы',
#             'type': 'pos',
#             'freq': None
#         },
#         {
#             'path1': '../dataset/Сокращение по частям речи/4.Первый жанр без клауз, включающих глаголы и наречия.txt',
#             'path2': '../dataset/Второй_жанр_исходная.txt',
#             'name': '4.Первый жанр без клауз, включающих глаголы и наречия',
#             'type': 'pos',
#             'freq': None
#         },
    ]

def main():

    results = []
    for model in models:
        for dataset in datasets:
            config = create_custom_config(model['model'], model['type'], dataset)
            
            
            # Entrenar y limpiar memoria
            torch.cuda.empty_cache()
            result = funciones.train_and_evaluate_dataset(
                dataset['path1'],
                dataset['path2'],
                config,
                dataset['name'],
                dataset['type']
                
            )
            results.append(result) 
        
        del model
        torch.cuda.empty_cache()
        gc.collect()
        
    
    save_results(results, 'model_results_GPT_bert_all.json')    

    
if __name__ == "__main__":
    main()

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


train_Изъяты лексемы с частотой выше 100.csv
test_Изъяты лексемы с частотой выше 100.csv
Train labels sample: tensor([0, 0, 0, 0, 1]), Shape: torch.Size([864])
Test labels sample: tensor([0, 0, 0, 0, 0]), Shape: torch.Size([195])
Epoch 1, Loss: 0.3086095005273819
Epoch 2, Loss: 0.03777603558929903
Epoch 3, Loss: 0.006088302271174533
Epoch 4, Loss: 0.005040814617781767
Epoch 5, Loss: 0.026641652736413692
Epoch 6, Loss: 0.00813552297352414
all_preds shape: (195,)
all_labels shape: (195,)
all_probs shape: (195, 2)
Train labels sample: tensor([1, 0, 1, 0, 0]), Shape: torch.Size([864])
Test labels sample: tensor([0, 0, 0, 0, 0]), Shape: torch.Size([198])
Epoch 1, Loss: 0.328420558146068
Epoch 2, Loss: 0.0938217458980424
Epoch 3, Loss: 0.03659572239433016
Epoch 4, Loss: 0.009948871131720287
Epoch 5, Loss: 0.003995593886689416
Epoch 6, Loss: 0.001113946333394519
all_preds shape: (198,)
all_labels shape: (198,)
all_probs shape: (198, 2)
Train labels sample: tensor([1, 0, 0, 0, 1]), Shape: torc

Epoch 1, Loss: 0.3739632357444082
Epoch 2, Loss: 0.046635067356484275
Epoch 3, Loss: 0.05151533581582563
Epoch 4, Loss: 0.021691589183839306
Epoch 5, Loss: 0.00633539109756904
Epoch 6, Loss: 0.004937135331731822
all_preds shape: (169,)
all_labels shape: (169,)
all_probs shape: (169, 2)
Train labels sample: tensor([1, 0, 1, 0, 0]), Shape: torch.Size([893])
Test labels sample: tensor([0, 0, 0, 0, 0]), Shape: torch.Size([164])
Epoch 1, Loss: 0.49733954455171314
Epoch 2, Loss: 0.09982437533991677
Epoch 3, Loss: 0.025377418180661544
Epoch 4, Loss: 0.016392703833324567
Epoch 5, Loss: 0.0037249907452080932
Epoch 6, Loss: 0.0024078608257696033
all_preds shape: (164,)
all_labels shape: (164,)
all_probs shape: (164, 2)
Train labels sample: tensor([1, 0, 1, 0, 0]), Shape: torch.Size([893])
Test labels sample: tensor([0, 0, 0, 0, 0]), Shape: torch.Size([164])
Epoch 1, Loss: 0.5543669249330249
Epoch 2, Loss: 0.09280082583427429
Epoch 3, Loss: 0.05177431067984019
Epoch 4, Loss: 0.016960939592016593


Epoch 1, Loss: 0.9040160434586662
Epoch 2, Loss: 0.6759007147380284
Epoch 3, Loss: 0.6704493590763637
Epoch 4, Loss: 0.6645495891571045
Epoch 5, Loss: 0.6620397567749023
Epoch 6, Loss: 0.6626296554292951
all_preds shape: (203,)
all_labels shape: (203,)
all_probs shape: (203, 2)
Train labels sample: tensor([0, 0, 0, 0, 0]), Shape: torch.Size([887])
Test labels sample: tensor([0, 0, 0, 0, 0]), Shape: torch.Size([199])
Epoch 1, Loss: 0.8801490494183132
Epoch 2, Loss: 0.684903587613787
Epoch 3, Loss: 0.6708846858569554
Epoch 4, Loss: 0.6669694270406451
Epoch 5, Loss: 0.660994325365339
Epoch 6, Loss: 0.6637258444513593
all_preds shape: (199,)
all_labels shape: (199,)
all_probs shape: (199, 2)
Train labels sample: tensor([0, 1, 0, 0, 0]), Shape: torch.Size([887])
Test labels sample: tensor([0, 0, 0, 0, 0]), Shape: torch.Size([194])
Epoch 1, Loss: 1.0147884828703744
Epoch 2, Loss: 0.6897607105118888
Epoch 3, Loss: 0.6736181463514056
Epoch 4, Loss: 0.6641272902488708
Epoch 5, Loss: 0.671034038

Epoch 1, Loss: 1.2131410221258798
Epoch 2, Loss: 0.5921114842096965
Epoch 3, Loss: 0.5892889738082886
Epoch 4, Loss: 0.5156688074270884
Epoch 5, Loss: 0.49319352904955543
Epoch 6, Loss: 0.476404865582784
all_preds shape: (162,)
all_labels shape: (162,)
all_probs shape: (162, 2)
Train labels sample: tensor([0, 0, 0, 0, 0]), Shape: torch.Size([900])
Test labels sample: tensor([0, 0, 0, 0, 0]), Shape: torch.Size([161])
Epoch 1, Loss: 0.6183641831080119
Epoch 2, Loss: 0.5665915111700693
Epoch 3, Loss: 0.5657343149185181
Epoch 4, Loss: 0.5572344402472178
Epoch 5, Loss: 0.515760354200999
Epoch 6, Loss: 0.4764257351557414
all_preds shape: (161,)
all_labels shape: (161,)
all_probs shape: (161, 2)
Train labels sample: tensor([0, 1, 1, 0, 0]), Shape: torch.Size([900])
Test labels sample: tensor([0, 0, 0, 0, 0]), Shape: torch.Size([158])
Epoch 1, Loss: 1.4152865211168926
Epoch 2, Loss: 0.6237733324368795
Epoch 3, Loss: 0.5173249026139577
Epoch 4, Loss: 0.5044663906097412
Epoch 5, Loss: 0.43728011

In [ ]:
#batch_size=128

In [ ]:
#display_results()

In [ ]:
import os
import IPython

# Reiniciar el kernel
IPython.display.display(IPython.display.Javascript("Jupyter.notebook.kernel.restart()"))

# Apagar el kernel después de reiniciar
os._exit(0)